<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3423e74f261906acc60ce2383c07609136ee85aab4a4824825f9326e392310be
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-17 15:17:42
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -14.08 K (-0.1%)
Current PnL: -21.23 L (-14.14%)
CY Booked + Current PnL: -9.33 L (-6.22%)
-------------------
Total profit:  1.82 L
Total loss:  -23.05 L
-------------------
Total Booked + Current PnL: 17.65 L (14.34%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.39 L (63.03%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.09,61.0,X-LC,3.89,216282.0,11782.0,11506.0,0.59,5.76,5.32,11.39,10.0,1.02,1.55,15.23,X40N,ATH,PHARMA
83,VOLTAS,1530.00,2.00,60.0,X-MC,1.34,214110.0,22368.0,15395.0,0.83,11.67,7.19,19.69,99.0,1.45,1.54,19.26,XY25,NTT,AC
76,TTKPRESTIG,770.00,95.09,46.0,M-SC,1.69,84036.0,-16741.0,16832.0,-1.93,-16.61,20.03,0.09,245.0,-0.99,0.60,10.41,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.79,63.0,X-LC,2.85,221052.0,10686.0,18104.0,1.34,5.08,8.19,13.68,37.0,0.59,1.59,22.44,XY25,NTT,REFINERIES
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.33,148148.0,12015.0,19111.0,-0.40,8.83,12.90,22.86,91.0,0.63,1.06,22.55,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.64,69.0,M-SC,4.17,103695.0,12196.0,66551.0,11.70,13.33,64.18,86.07,223.0,0.18,0.75,50.58,XR,NTT,DURABLES
25,GREENPANEL,537.00,241.73,60.0,M-SC,4.28,152598.0,-30480.0,109459.0,5.13,-16.65,71.73,43.14,230.0,-0.28,1.10,43.54,XY24,NTT,MISC
45,KPIGREEN,731.05,1.34,59.0,H-SC,6.15,120607.0,-4690.0,63620.0,4.10,-3.74,52.75,47.03,141.0,-0.07,0.87,51.01,MH,ATH,POWER
5,ASIANPAINT,3465.66,-12.54,65.0,X-LC,5.30,220686.0,-31082.0,84302.0,4.07,-12.35,38.20,21.13,27.0,-0.37,1.59,18.71,X40,ATH,PAINTS
67,STARHEALTH,761.00,23.71,73.0,H-SC,9.27,275454.0,-5669.0,138526.0,3.24,-2.02,50.29,47.26,144.0,-0.04,1.98,47.58,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-15.15,43.0,M-LC,5.71,149439.0,-1506.0,110958.0,-5.04,-1.00,74.25,72.51,53.0,-0.01,1.07,4.96,XR,NTT,IT
70,TANLA,1963.11,-23.20,44.0,H-SC,19.65,206305.0,-64456.0,394414.0,-4.73,-23.81,191.18,121.86,133.0,-0.16,1.48,61.39,AR,ATH,IT
63,SHALBY,327.00,1155.18,51.0,M-SC,23.28,170721.0,-10422.0,55894.0,-2.53,-5.75,32.74,25.10,235.0,-0.19,1.23,36.82,XY24,NTT,HEALTHCARE
50,PGHH,17907.65,-28.69,46.0,X-MC,3.32,206250.0,5430.0,62370.0,-2.37,2.70,30.24,33.76,80.0,0.09,1.48,8.04,X40,ATH,FMCG
7,ATULAUTO,844.00,3618.92,49.0,M-SC,3.74,167110.0,-30022.0,119851.0,-2.33,-15.23,71.72,45.57,236.0,-0.25,1.20,18.48,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,18.70,150255.0,1035.0,143148.0,0.04,0.69,95.27,96.62,208.0,0.01,1.08,62.61,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.39,39.0,H-SC,7.91,124391.0,1055.0,130573.0,-0.39,0.86,104.97,106.72,119.0,0.01,0.89,24.43,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,12.32,131436.0,-11043.0,30559.0,-0.63,-7.75,23.25,13.70,163.0,-0.36,0.94,55.13,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,61.0,H-SC,1.90,227772.0,-41895.0,78217.0,3.23,-15.54,34.34,13.47,138.0,-0.54,1.64,15.82,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.09,46.0,M-SC,1.69,84036.0,-16741.0,16832.0,-1.93,-16.61,20.03,0.09,245.0,-0.99,0.60,10.41,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.11,39.0,H-SC,2.36,138230.0,-37673.0,79690.0,-0.48,-21.42,57.65,23.89,149.0,-0.47,0.99,19.38,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,44.0,H-MC,7.34,102030.0,-29805.0,72033.0,-0.62,-22.61,70.60,32.03,98.0,-0.41,0.73,14.28,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,50.0,M-LC,6.22,206388.0,3218.0,38904.0,-0.60,1.58,18.85,20.74,55.0,0.08,1.48,7.21,XY25,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,9.98,217674.0,12598.0,99978.0,-1.24,6.14,45.93,54.89,52.0,0.13,1.56,31.72,X5K,ATH,METALS
78,UNIONBANK,163.00,-14.73,50.0,M-LC,8.74,155496.0,14656.0,29840.0,-1.48,10.41,19.19,31.59,66.0,0.49,1.12,39.65,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189405.0,9597.0,102809.0,-1.81,5.34,54.28,62.52,88.0,0.09,1.36,41.24,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-38.64,69.0,M-SC,4.17,103695.0,12196.0,66551.0,11.70,13.33,64.18,86.07,223.0,0.18,0.75,50.58,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.64,69.0,M-SC,4.17,103695.0,12196.0,66551.0,11.70,13.33,64.18,86.07,223.0,0.18,0.75,50.58,XR,NTT,DURABLES
31,HINDZINC,730.22,33.63,61.0,M-LC,9.98,217674.0,12598.0,99978.0,-1.24,6.14,45.93,54.89,52.0,0.13,1.56,31.72,X5K,ATH,METALS
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189405.0,9597.0,102809.0,-1.81,5.34,54.28,62.52,88.0,0.09,1.36,41.24,XR,NTT,BANKS
36,INDIAMART,4810.62,-55.39,39.0,H-SC,7.91,124391.0,1055.0,130573.0,-0.39,0.86,104.97,106.72,119.0,0.01,0.89,24.43,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,5.71,149439.0,-1506.0,110958.0,-5.04,-1.00,74.25,72.51,53.0,-0.01,1.07,4.96,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-54.49,14.0,X-LC,7.45,144147.0,-132376.0,242441.0,0.08,-47.87,168.19,39.81,54.0,-0.55,1.04,1.60,XY24,NTT,AUTO
51,QUESS,986.0,-48.76,32.0,X-SC,38.69,51147.0,-13859.0,164785.0,-2.01,-21.32,322.18,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
39,INFY,2275.0,-21.03,40.0,X-LC,7.49,306976.0,-5912.0,177586.0,-2.06,-1.89,57.85,54.87,3.0,-0.03,2.21,4.63,X40,BTT,IT
12,BATAINDIA,2096.0,-36.15,42.0,X-SC,4.31,95508.0,-33162.0,78460.0,0.26,-25.77,82.15,35.20,219.0,-0.42,0.69,9.62,X40,NTT,FOOTWEAR
55,RELAXO,1176.0,-39.78,42.0,X-SC,4.42,82118.0,-63042.0,138968.0,-1.00,-43.43,169.23,52.31,136.0,-0.45,0.59,9.92,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.40,156105.0,-29990.0,77475.0,0.10,-16.12,49.63,25.51,15.0,-0.39,1.12,15.04,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.00,60.0,X-MC,1.34,214110.0,22368.0,15395.0,0.83,11.67,7.19,19.69,99.0,1.45,1.54,19.26,XY25,NTT,AC
28,HAVELLS,2069.16,-6.18,44.0,X-MC,2.09,232658.0,-15073.0,90132.0,1.06,-6.08,38.74,30.30,92.0,-0.17,1.67,6.40,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.33,148148.0,12015.0,19111.0,-0.40,8.83,12.90,22.86,91.0,0.63,1.06,22.55,X40,ATH,INSURANCE
41,ITC,452.00,-37.50,59.0,X-LC,2.64,201444.0,1306.0,19580.0,1.68,0.65,9.72,10.44,4.0,0.07,1.45,6.25,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,32.0,X-SC,38.69,51147.0,-13859.0,164785.0,-2.01,-21.32,322.18,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.45,144147.0,-132376.0,242441.0,0.08,-47.87,168.19,39.81,54.0,-0.55,1.04,1.60,XY24,NTT,AUTO
73,TCS,4389.97,-29.97,45.0,X-LC,12.64,278531.0,-67425.0,134113.0,-0.26,-19.49,48.15,19.28,1.0,-0.50,2.00,2.97,X40,ATH,IT
2,ACC,3906.00,-38.07,45.0,X-MC,3.27,183690.0,-54261.0,206908.0,-1.25,-22.80,112.64,64.15,174.0,-0.26,1.32,3.04,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179449.0,-21057.0,118203.0,-0.56,-10.50,65.87,48.45,90.0,-0.18,1.29,3.48,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,32.0,X-SC,38.69,51147.0,-13859.0,164785.0,-2.01,-21.32,322.18,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.78,42.0,X-SC,4.42,82118.0,-63042.0,138968.0,-1.00,-43.43,169.23,52.31,136.0,-0.45,0.59,9.92,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.15,42.0,X-SC,4.31,95508.0,-33162.0,78460.0,0.26,-25.77,82.15,35.20,219.0,-0.42,0.69,9.62,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,45.0,X-SC,0.86,106268.0,4264.0,39659.0,-0.93,4.18,37.32,43.06,122.0,0.11,0.76,22.01,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.51,108900.0,-19038.0,66168.0,0.58,-14.88,60.76,36.84,143.0,-0.29,0.78,11.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.64,278531.0,-67425.0,134113.0,-0.26,-19.49,48.15,19.28,1.0,-0.50,2.00,2.97,X40,ATH,IT
39,INFY,2275.00,-21.03,40.0,X-LC,7.49,306976.0,-5912.0,177586.0,-2.06,-1.89,57.85,54.87,3.0,-0.03,2.21,4.63,X40,BTT,IT
41,ITC,452.00,-37.50,59.0,X-LC,2.64,201444.0,1306.0,19580.0,1.68,0.65,9.72,10.44,4.0,0.07,1.45,6.25,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.53,294278.0,-21564.0,134220.0,-0.01,-6.83,45.61,35.67,5.0,-0.16,2.11,6.08,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,50.0,H-LC,7.08,249744.0,-11875.0,131091.0,-0.17,-4.54,52.49,45.57,7.0,-0.09,1.79,6.27,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6794.44,59.0,L-SC,6.26,75673.0,-18137.0,94758.0,0.70,-19.33,125.22,81.67,269.0,-0.19,0.54,46.54,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,29.16,81408.0,-32141.0,72193.0,1.56,-28.31,88.68,35.27,268.0,-0.45,0.58,99.25,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.45,89190.0,-8790.0,30396.0,-1.83,-8.97,34.08,22.05,152.0,-0.29,0.64,29.64,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-38.64,69.0,M-SC,4.17,103695.0,12196.0,66551.0,11.70,13.33,64.18,86.07,223.0,0.18,0.75,50.58,XR,NTT,DURABLES
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,12.32,131436.0,-11043.0,30559.0,-0.63,-7.75,23.25,13.70,163.0,-0.36,0.94,55.13,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,18.70,150255.0,1035.0,143148.0,0.04,0.69,95.27,96.62,208.0,0.01,1.08,62.61,XY25,NTT,FINANCE
67,STARHEALTH,761.00,23.71,73.0,H-SC,9.27,275454.0,-5669.0,138526.0,3.24,-2.02,50.29,47.26,144.0,-0.04,1.98,47.58,XY24,NTT,INSURANCE
4,ANGELONE,3033.00,15.82,69.0,X-SC,8.84,204623.0,13617.0,44076.0,0.52,7.13,21.54,30.21,157.0,0.31,1.47,29.50,X40N,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,9.98,217674.0,12598.0,99978.0,-1.24,6.14,45.93,54.89,52.0,0.13,1.56,31.72,X5K,ATH,METALS
63,SHALBY,327.00,1155.18,51.0,M-SC,23.28,170721.0,-10422.0,55894.0,-2.53,-5.75,32.74,25.10,235.0,-0.19,1.23,36.82,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,44.25
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.33
LC    33.83
MC    21.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.62
X40      14.80
XY25     12.41
XR       10.90
X40N     10.23
AR        8.07
OX40N     7.60
X200      1.81
X5K       1.56
SR        1.12
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.39
X-LC    22.85
X-MC    16.14
M-SC    12.82
M-LC     5.23
H-LC     4.72
X-SC     4.66
H-MC     3.78
M-MC     1.60
L-SC     1.46
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.30,-3.09,37.07
IT,12.70,-18.87,83.03
FINANCE,12.14,-11.82,62.51
MISC,6.90,-19.86,86.28
BANKS,6.31,-13.66,73.45
PAINTS,5.73,-15.92,33.20
ELECTRICAL,5.36,-10.17,48.82
INSURANCE,4.01,0.92,36.12
AUTO,2.82,-46.56,116.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3317127.0,22
XR,1392283.0,14
AR,1206197.0,9
X40,778215.0,10
X40N,601547.0,8
OX40N,587074.0,10
XY25,455329.0,8
SR,264796.0,2
X5K,99978.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3466001.0,25
M-SC,1258922.0,15
X-MC,1080563.0,11
X-LC,1057936.0,13
X-SC,532116.0,6
H-MC,353894.0,3
H-LC,281823.0,3
M-LC,279680.0,4
L-SC,257184.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190976.0      6
           XR         835561.0      7
           AR         835306.0      5
M-SC       XY24       688281.0      6
X-MC       XY24       584503.0      4
X-LC       X40        484177.0      5
           XY24       309132.0      2
X-SC       X40N       288871.0      4
H-SC       OX40N      275742.0      4
           SR         264796.0      2
X-MC       X40        215578.0      4
H-LC       AR         208566.0      2
X-LC       X40N       194473.0      3
H-MC       XY24       179052.0      1
M-MC       XY24       170558.0      1
L-SC       XR         166951.0      2
X-SC       XY24       164785.0      1
M-SC       AR         162325.0      2
X-MC       XY25       162279.0      2
M-SC       OX40N      149066.0      4
           XY25       143148.0      1
X-MC       X40N       118203.0      1
M-SC       XR         116102.0      2
M-LC       XR         110958.0      1
H-MC       XR         102809.0      1
M-LC       X5K         99978.0      1
L-SC       OX40N       90233.0      1
X-SC       X40         78460.0      1
H-LC       X200        73257.0      1
H-MC       OX40N       72033.0      1
X-LC       XY25        70154.0      3
H-SC       MH          63620.0      1
L-MC       XR          59902.0      1
L-LC       XY25        40844.0      1
M-LC       XY25        38904.0      1
           XY24        29840.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
